In [ ]:
# 1. 우량후보 여부: 한도 5만 원 미만(C,D,E)은 0, 이상은 1 (1차 필터)
df['우량후보_여부'] = (df['카드이용한도금액'] >= 50000).astype(int)

# 2. 실질 여유 한도: 한도는 크고 소진율은 낮은 '찐부자(A)'를 가려내는 변수
df['실질여유한도'] = df['카드이용한도금액'] * (1 - df['잔액_신판최대한도소진율_r3m'])

# 3. 한도 대비 소비성향: A(안정)와 B(공격적)를 가르고, CDE(한도부족)를 99로 밀어내는 변수
df['한도대비_소비성향'] = df['잔액_신판최대한도소진율_r3m'] / np.log1p(df['카드이용한도금액'])

# (중요) 한도가 0에 가까워 무한대(inf)가 나오면 하위 등급 값(99)으로 처리
df['한도대비_소비성향'] = df['한도대비_소비성향'].replace([np.inf, -np.inf], 99).fillna(99)

In [ ]:
# 4. 장기 우량 고객 (한도 10만 & 입회 10년 이상 가정)
# '입회경과개월수_신용'이 120개월 이상인 사람
df['장기 우량 고객'] = ((df['카드이용한도금액'] >= 100000) & (df['입회경과개월수_신용'] >= 120)).astype(int)

# 5. 클린 신용 지표 (최근 1년 카드론/CA 이용 0건)
df['클린 신용 지표'] = ((df['이용건수_카드론_R12M'] == 0) & (df['이용건수_CA_R12M'] == 0)).astype(int)

# 6. 할부 의존 지표 (일시불 잔액보다 할부 잔액이 많은 경우)
df['할부 의존 지표'] = (df['잔액_할부_B0M'] > df['잔액_일시불_B0M']).astype(int)

In [ ]:
numeric_cols = ['잔액_리볼빙일시불이월_B0M', '잔액_리볼빙CA이월_B0M', '변동률_잔액_B1M', '혜택수혜율_R3M']

for col in numeric_cols:
    if col in df.columns:
        # 문자열을 숫자로 변환 (오류 방지 핵심)
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# 7. 리볼빙_사용여부 (자금 압박 상태 확인)
df['리볼빙_사용여부'] = ((df['잔액_리볼빙일시불이월_B0M'] > 0) | (df['잔액_리볼빙CA이월_B0M'] > 0)).astype(int)

# 8. 잔액_급증신호 (단기 부채 급증 감지)
df['잔액_급증신호'] = (df['변동률_잔액_B1M'] > 0.5).astype(int)

# 9. 혜택_고수혜여부 (체리피커 성향 확인)
benefit_mean = df['혜택수혜율_R3M'].mean()
df['혜택_고수혜여부'] = (df['혜택수혜율_R3M'] > benefit_mean).astype(int)